In [1]:
%run supportvectors-common.ipynb



<div style="color:#aaa;font-size:8pt">
<hr/>

 </blockquote>
 <hr/>
</div>



# Vector database for ANN

In this lab, we will learn to use a vector database to store our vectors, and perform approximate nearest neighbor searches.

Run the below installation line only once (uncomment it first).

In [2]:
#!pip install chromadb

Let us now create a database client, and use it to instantiate a new collection instance. A collection is, approximately  like a schema-instance in a relational database: in other words, it is a logical and coherent collection or pile of related data.


In [3]:
import chromadb
client = chromadb.Client()
client.create_collection(name="medical")


Using embedded DuckDB without persistence: data will be transient
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


Collection(name=medical)

It seems we may loose all our data! We forgot to make it persistent. Let us redo this, with the additional parameters.

In [13]:
from chromadb.config import Settings
client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet",
                                    persist_directory="/home/asif/chromabd/data"
                                ))
collection = client.get_or_create_collection(name="medical")

Using embedded DuckDB with persistence: data will be stored in: /home/asif/chromabd/data
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


To check if the database is alive and doing well, let's try a hearbeat function on it.

In [14]:
client.heartbeat()

1685061750249162653000

There are four important methods that pertain to a collection:

* `list_collections()`  will list all the collections present int eh database
* `create_collection('<name-of-collection>')` will create a new collection with that name
* `get_collection('<name-of-collection>')` will return a `Collection` object, if such a collection exists
* `get_or_create_collection('<name-of-collection>')` will also return a `Collection` object. If a collection by that name does not exist, it will create it first
* `delete_collection('<name-of-collection>')` will delete the collection with that given name

## Methods associated with a collection

Some of the methods associated with a collection are:

* `modify(name='<new-name>')` will rename the collection
* `count()` will return the size of the collection, i.e., the number of items or records in that collection
* `add(...)` to add items to the collection
* `get()` will retrieve the items in the collection
* `peek()` a convenience method to get the first 5 items of the collection
* `query()` the main method perform the k-nearest neighbors search
* `delete()` to delete all the items in the collection.

## Building a approximate nearest neighbors search index
Let us start by creating a small search index.

In [15]:
%run Lesson_34___corpus.ipynb

In [16]:
from IPython.display import Markdown, HTML
for sentence in sentences:
    print('_'*100)
    display(Markdown(sentence))

____________________________________________________________________________________________________



’Twas brillig, and the slithy toves
      Did gyre and gimble in the wabe:
All mimsy were the borogoves,
      And the mome raths outgrabe.

“Beware the Jabberwock, my son!
      The jaws that bite, the claws that catch!
Beware the Jubjub bird, and shun
      The frumious Bandersnatch!”

He took his vorpal sword in hand;
      Long time the manxome foe he sought—
So rested he by the Tumtum tree
      And stood awhile in thought.

And, as in uffish thought he stood,
      The Jabberwock, with eyes of flame,
Came whiffling through the tulgey wood,
      And burbled as it came!

One, two! One, two! And through and through
      The vorpal blade went snicker-snack!
He left it dead, and with its head
      He went galumphing back.

“And hast thou slain the Jabberwock?
      Come to my arms, my beamish boy!
O frabjous day! Callooh! Callay!”
      He chortled in his joy.

’Twas brillig, and the slithy toves
      Did gyre and gimble in the wabe:
All mimsy were the borogoves,
      And the mome raths outgrabe.



____________________________________________________________________________________________________



It was the best of times, it was the worst of times, 
it was the age of wisdom, it was the age of foolishness, 
it was the epoch of belief, it was the epoch of incredulity, 
it was the season of light, it was the season of darkness, 
it was the spring of hope, it was the winter of despair, 
we had everything before us, we had nothing before us, 
we were all going direct to heaven, 
we were all going direct the other way–in short, 
the period was so far like the present period, 
that some of its noisiest authorities insisted on its being received, 
for good or for evil, in the superlative degree of comparison only.


____________________________________________________________________________________________________


...


In machine learning, backpropagation
(backprop,[1] BP) is a widely used
algorithm for training feedforward
artificial neural networks.
Generalizations of backpropagation
exist for other artificial neural
networks (ANNs), and for functions
generally. These classes of algorithms
are all referred to generically as
"backpropagation".[2] In fitting a
neural network, backpropagation
computes the gradient of the loss
function with respect to the weights of
the network for a single input–output
example, and does so efficiently,
unlike a naive direct computation of
the gradient with respect to each
weight individually. This efficiency
makes it feasible to use gradient
methods for training multilayer
networks, updating weights to minimize
loss; gradient descent, or variants
such as stochastic gradient descent,
are commonly used. The backpropagation
algorithm works by computing the
gradient of the loss function with
respect to each weight by the chain
rule, computing the gradient one layer
at a time, iterating backward from the
last layer to avoid redundant
calculations of intermediate terms in
the chain rule; this is an example of
dynamic programming.[3]


____________________________________________________________________________________________________



She dwelt among the untrodden ways
Beside the springs of Dove,
A Maid whom there were none to praise
And very few to love:

A violet by a mossy stone
Half hidden from the eye!
—Fair as a star, when only one
Is shining in the sky.

She lived unknown, and few could know
When Lucy ceased to be;
But she is in her grave, and, oh,
The difference to me!



____________________________________________________________________________________________________



What is this life if, full of care,
We have no time to stand and stare.

No time to stand beneath the boughs
And stare as long as sheep or cows.

No time to see, when woods we pass,
Where squirrels hide their nuts in grass.

No time to see, in broad daylight,
Streams full of stars, like skies at night.

No time to turn at Beauty's glance,
And watch her feet, how they can dance.

No time to wait till her mouth can
Enrich that smile her eyes began.

A poor life this if, full of care,
We have no time to stand and stare.

 


### Add it to a collection

Let us consider each of these sentences as a **document**. We need a unique identifier for each document. Towards this, let us associate one with each:

In [22]:
ids = []
for index, _ in enumerate(sentences):
    ids.append(str(index))
ids

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15']

In [23]:
collection.add(documents=sentences, ids=ids)

In [24]:
query_text = "a friendship with animals"
results = collection.query(query_texts = [query_text],
                          n_results=3)
results

{'ids': [['7', '9', '5']],
 'embeddings': None,
 'documents': [['\nGolden retrievers are not bred to be guard dogs, and considering the size of their hearts and their irrepressible joy and life, they are less likely to bite than to bark, less likely to bark than to lick a hand in greeting. In spite of their size, they think they are lap dogs, and in spite of being dogs, they think they’re also human, and nearly every human they meet is judged to have the potential to be a boon companion who might at any moment, cry, “Let’s go!” and lead them on a great adventure.\n',
   '\nMy friend Phil has a theory that the Lord, having made teenagers, felt constrained to make amends and so created the golden retriever.\n',
   'If a man aspires towards a righteous life, his first act of abstinence is from injury to animals.']],
 'metadatas': [[None, None, None]],
 'distances': [[1.288403868675232, 1.3694041967391968, 1.3841540813446045]]}